### Adjusting the address for coordinate transformation

In [2]:
import pandas as pd
import os
import numpy as np
data=pd.read_csv('/Users/kazba1/Downloads/combined_with_coordinates.csv')
df=pd.DataFrame(data)
for i in range(len(df.index)):
    try:
        df.loc[i,'short_adress']=df.loc[i,'links_Appartment_Address'].partition("-")[0]
    except AttributeError:
        pass
    try:
        df.loc[i,'short_adress']=df.loc[i,'short_adress'].partition("SV")[0]
    except AttributeError:
        pass
    try:
        df.loc[i,'short_adress']=df.loc[i,'short_adress'].partition("NV")[0]
    except AttributeError:
        pass
    try:
        df.loc[i,'short_adress']=df.loc[i,'short_adress'].partition("C")[0]
    except AttributeError:
        pass
    try:
        str = df.loc[i,'short_adress'];
        df.loc[i,'short_adress'] = str.replace("Valby", "Copenhagen");
        df.loc[i,'short_adress'] = str.replace("Brønshøj", "Copenhagen");
        df.loc[i,'short_adress'] = str.replace("Nordvest", "Copenhagen");
    except AttributeError:
        pass

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 21 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        2900 non-null   int64  
 1   links_Appartment_url              2900 non-null   object 
 2   links_Appartment_Name             2900 non-null   object 
 3   links_Appartment_Address          2900 non-null   object 
 4   links_Appartment_Boligtype        2900 non-null   object 
 5   links_Appartment_Rooms            2900 non-null   float64
 6   links_Appartment_Floor            2900 non-null   object 
 7   links_Appartment_Furnished        2900 non-null   object 
 8   links_Appartment_Shareable        2900 non-null   object 
 9   links_Appartment_Pets_allowed     2900 non-null   object 
 10  links_Appartment_Rent_period      2900 non-null   object 
 11  links_Appartment_Take_over_date   2900 non-null   object 
 12  links_

### Using Geopy to transform addresses to coordinates

In [ ]:
>>> from geopy.geocoders import Nominatim
>>> geolocator = Nominatim(user_agent='robertas.kazbaras@gmail.com')
for i in range(len(df.index)):
    try: 
        location = geolocator.geocode(df.loc[i,'short_adress'])
        df.loc[i,'latitude']=location.latitude
        df.loc[i,'longitude']=location.longitude
    except AttributeError:
        pass

### Calculating the distance to CBS

In [ ]:
#Taking non nul coordinates
dataset_with_missing_coordinates=df[df['latitude'].isnull()==False]

#Using the geopy for CBS coordinates
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='robertas.kazbaras@gmail.com')
location = geolocator.geocode("Solbjerg Pl. 3, 2000, Copenhagen")
coords_1=(location.latitude,location.longitude)

#Calculating the distance
import geopy.distance
for i in range(len(dataset_with_missing_coordinates.index)):
    coords_2 = (dataset_with_missing_coordinates.loc[i,"latitude"],dataset_with_missing_coordinates.loc[i,"longitude"])
    dataset_with_missing_coordinates.loc[i,"distance to CBS"]= geopy.distance.geodesic(coords_1, coords_2).km

In [ ]:
dataset_with_missing_coordinates

In [ ]:
#Saving the csv
dataset_with_missing_coordinates.to_csv('/Users/kazba1/Downloads/with_distance_to_CBS.csv')